In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math as mt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from hangul_utils import split_syllables, join_jamos
from tqdm.auto import tqdm
import re
#from eunjeon import Mecab
from konlpy.tag import Mecab
from hanspell import spell_checker

In [2]:
con_dict = [
    
    ['ㅏㅣ','ㅐ'], ['ㅑㅣ','ㅒ'], ['ㅓㅣ','ㅔ'],
    ['ㅕㅣ','ㅖ'], ['ㅗㅣ','ㅚ'], ['ㅗㅐ','ㅙ'],
    ['ㅜㅓ','ㅝ'], ['ㅜㅔ','ㅞ'], ['ㅡㅣ','ㅢ'],
    ['ㅣㅏ','ㅑ'], ['ㅣㅓ','ㅕ'], ['ㅣㅗ','ㅛ'],
    ['ㅣㅜ','ㅠ'], ['ㅡㅓ','ㅓ'], ['ㅗㅏ','ㅘ']
    
]

jongsung_list = [ 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


#lis_beta = ['EP+EF', 'VCP+EF', 'B+EF', 'B+EP+EF', 'B+VCP+EF', 'EF','EP']

lis_beta = ['EP+EF', 'EF', 'B+EF', 'B+EP+EF']

#어말을 처리해 주기 위한 것으로, 나중에 EC등이 필요해 진다면 이 부분에 EC 등을 집어넣어준다. 참고로 말하자면 이는 마지막에 위치해야한다.
#특이 바로 밑의 이 부분은 형태소 태그가 이 리스트 안의 것과 일치하는 경우 단순 삭제를 하는 것이고
lis_beta_ef = ['EP+EF', 'EF', 'UNKNOWN']

#이 부분 같은 경우는 마지막에 오는 것을 처리하는 것으로 단독으로만 들어가는게 좋겠지?
lis_tag_last = ['EF', 'UNKNOWN']

lis_end = [
    
    'ㅂㄴㅣㄷㅏ',
    'ㅅㅔㅇㅛ', 'ㄷㅔㅇㅛ', 'ㅇㅔㅇㅛ', 'ㅇㅖㅇㅛ', 'ㄴㅏㅇㅛ', 'ㅇㅡㄹㄲㅏㅇㅛ', 'ㅇㅣㄹㄲㅏㅇㅛ', 'ㄹㄲㅏㅇㅛ', 'ㅇㅡㄴㄱㅏㅇㅛ', 'ㅇㅣㄴㄱㅏㅇㅛ','ㅇㅛ',
    'ㅈㅛ',
    'ㅅㅣㅂㅅㅣㅇㅗ', 'ㅅㅣㅇㅗ', 'ㅇㅗ',
    'ㅂㄴㅣㄲㅏ', 'ㅅㅡㅂㄴㅣㄷㅏ', 'ㄴㅣㄲㅏ', 'ㄲㅏ', 
    
]

lis_end_2low = [
    
    'ㄷㅓㄹㅏ','ㄴㄷㅏ', 'ㅆㄷㅏ', 'ㄹㅗㄷㅏ', 'ㄷㅏ', 
    'ㄹㅏ',
    'ㅇㅑ', 
    'ㄲㅏ', 
    'ㄴㅣ', 
    'ㅇㅏ', 
    
]

# lis_end_2low = [
    
#     'ㄷㅓㄹㅏ','ㄴㄷㅏ', 'ㅆㄷㅏ', 'ㄷㅏ', 
#     'ㄹㅏ',
#     'ㅇㅑ', 
    
# ]

P_LIST = ['.', '?', '!']

SV_LIST = ['\'', '\"', ':', ';']

lis_plus = [
    
    'EP', 'VCP', 
    
]

In [3]:
mec = Mecab()

In [4]:
def unite(input, dict):
    for i in dict:
        input = re.sub(i[0],i[1],input)
    return input
    
## 자모 단위로 문장을 나누고 합칠 때 쓰는 class ##
class Jamodealer:
    jamo = []
    pp = ''
    #각 단어들을 받아와서 자모단위로 나눈다.
    def __init__(self,lis_word):
    
        self.jamo = []
        for i in lis_word:
            self.jamo.append(split_syllables(i))
    
    ##사전에서 변환된 자모단위로 분리된 문장을 합칠 때 쓰는 함수이다.     
    def make_one(self):
        #list 형태로 저장된 자모들의 집합을 하나의 string pp에 저장한다. 
        self.pp = ''
        for i in self.jamo:
             self.pp= self.pp+i
        ##종성과 종성을 합쳐야 하는 경우가 있다면 합친다.        
        self.pp = unite(self.pp, con_dict)
        
        #자모 단위의 string에서 자모 단위로 사전을 만들고 거기에 index를 부여한다.        
        chars = list(set(self.pp))
        char_to_ix = { ch:i for i,ch in enumerate(chars) }
        ix_to_char = { i:ch for i,ch in enumerate(chars) }
        
        #자모 단위로 분리되었던 문장을 다시 하나로 합친다.
        jamo_numbers = [char_to_ix[x] for x in self.pp]
        restored_jamo = ''.join([ix_to_char[x] for x in jamo_numbers])
        #합쳐진 문장을 return 한다.
        restored_text = join_jamos(restored_jamo)
        return restored_text

def to2lists(input):
    lis_word = []
    lis_tag = []
    #data = han.pos(input,ntags=22,flatten=True, join=False)
    data = mec.pos(input)
    for i in data:
        lis_word.append(i[0])
        lis_tag.append(i[1])
    return lis_word, lis_tag

def rememberSpace(lis, input):
    
    rlis = []
    
    for i in range(len(lis)):
        if lis[i]==input:
            rlis.append(i)
            
    for i in range(len(rlis)):
        rlis[i] = rlis[i]-i      
    return rlis

def convertSpace(lis_space,lis_lis):
    
    rlis = []
    k=0
    for i in range(len(lis_lis)):
        
        if k in lis_space:
            rlis.append(i)
            
        k = k+len(lis_lis[i])
        
    #print(rlis)  
    return rlis

def union(lis, lis_lis):
    
    k = 0
    for i in lis:
        lis_lis.insert(i+k,' ')
        k = k+1
 

In [5]:
def union_t(lis_lis):
    
    for i in range(len(lis_lis)-1):
        if lis_lis[i+1] != ' ' and lis_lis[i+1] !='SF' and lis_lis[i] != ' ' and lis_lis[i] !='SF':
            lis_lis[i] = lis_lis[i]+'/'
        if lis_lis[i+1]=='SF':
            lis_lis[i+1] = ' '+lis_lis[i+1]
            
def union_w(lis_w, lis_tag):
    
    for i in range(len(lis_w)-1):
        if lis_tag[i+1]==' SF':
            lis_w[i+1] = ' '+lis_w[i+1]

In [6]:
def union_t_03(lis_tag):
    
    for i in range(1, len(lis_tag)):
        if lis_tag[i-1] ==' ' or lis_tag[i]==' ':
            lis_tag[i] = lis_tag[i]
        else:
            lis_tag[i] = '/'+lis_tag[i]
            
def union_w_03(lis_w, lis_tag):
    
    for i in range(1, len(lis_w)):
        if lis_tag[i]==' SF':
            lis_w[i] = ' '+lis_w[i+1]

In [7]:
def prepro(input):
    lis_w, lis_t = to2lists(input)

    space_list = rememberSpace(input,' ')
    space_location = convertSpace(space_list, lis_w)
    union(space_location, lis_w)
    union(space_location, lis_t)
    union_t_03(lis_t)
    union_w_03(lis_w, lis_t)
    
    str_w = ''
    str_t = ''
    for i in range(len(lis_w)):
        str_w = str_w + lis_w[i]
        str_t = str_t + lis_t[i]
    
    data_w = str_w.split(' ')
    data_t = str_t.split(' ')
    
    lis_word, lis_tag = to2lists(input)
    
    lis_ind = []
    t_ind = 0
    jam1 = Jamodealer(lis_word)
    jam2 = Jamodealer(data_w)
    for i in range(len(data_w)):
        element = []
        leng = len(data_t[i].split('/'))
        res = jam2.jamo[i]
        ind = 0
        lenlen = 0
        #element.append(0)
        for j in range(leng):
            element.append(ind)
            ind = ind + len(jam1.jamo[t_ind])
            res = res[len(jam1.jamo[t_ind]):]
            
            lenlen = len(jam1.jamo[t_ind])+lenlen
            t_ind = t_ind+1

        element.append(len(jam2.jamo[i]))
        lis_ind.append(element)
        
    return data_w, data_t, lis_ind

In [8]:
prepro('백화점은 문을 닫았고 일자리는 구하기 힘들어 젔다? ')

(['백화점은', '문을', '닫았고', '일자리는', '구하기', '힘들어', '젔다?'],
 ['NNG/JX', 'NNG/JKO', 'VV/EP/EC', 'NNG/JX', 'VV/ETN', 'VA/EC', 'UNKNOWN/SF'],
 [[0, 8, 11],
  [0, 3, 6],
  [0, 3, 6, 8],
  [0, 7, 10],
  [0, 4, 6],
  [0, 6, 8],
  [0, 5, 6]])

In [9]:
def prepro_beta(input, lis_ef, tag_last, lis_w_last):
    data_w, data_t, lis_ind = prepro(input)
    
    last_words = []
    
    data_w_jamo = []
    
    data_t_after = []
    
    for i in data_w:
        jam_ele = Jamodealer(i)
        ele = ''
        for j in jam_ele.jamo:
            ele = ele+j
        data_w_jamo.append(ele)
    
    for i in range(len(data_t)):
        #if i<len(data_t)-1:
        if i<len(data_t):
            lis_res = []
            for ind in range(len(lis_ind[i])-1):
                lis_res.append(data_w_jamo[i][lis_ind[i][ind]:lis_ind[i][ind+1]])

        if 'EF' in data_t[i] or 'UNKNOWN' in data_t[i]:
            elements = data_t[i].split('/')
            
            flag = 0
            print(elements)
            for j in range(len(elements)):
                
                if elements[j] in lis_ef: #and j == len(elements)-1:
                    
                    elements[j] = 'NULL'
                    
                    last_words.append(data_w_jamo[i][lis_ind[i][j]:lis_ind[i][j+1]])
                    #lis_res[j] = data_w_jamo[i][:lis_ind[i][j]]
                    lis_res[j]=''
                    
                elif 'EF' in elements[j]:# + EF를 처리하는 부분이므로 + EF 만을 마지막에서 처리한다.
#                     res1 = data_w_jamo[i][:lis_ind[i][j+1]]
#                     res2 = data_w_jamo[i][lis_ind[i][j+1]:]
#                     print(res2)
#                     print(lis_ind[i][j+1])
                    for jam in lis_w_last:
#                         print('ㅈㅈ')
                        if len(lis_res[j])>=len(jam):
                            #if jam in lis_res[j]:
#                             print(jam)
                            print('ㅈㅈㅈㅈ')
                            if lis_res[j].find(jam)!=-1:
                                last_words.append(jam)
                                lis_res[j] = lis_res[j].replace(jam, '', 1)

                                for k in tag_last:

                                    if k in elements[j] and j==len(elements)-1:
                                        if '+' in elements[j]:
                                            ind = elements[j].index('+'+k)
                                            elements[j] = elements[j][:ind]
                                break

            data_w_jamo[i] = ''.join(lis_res)

            elements_post = '/'.join(elements)
            data_t_after.append(elements_post)
            
        else:
            data_t_after.append(data_t[i])
        
    return data_w, data_t, lis_ind, data_w_jamo, data_t_after, last_words
                        
                    
            

In [10]:
t1, t2, t3, t4, t5, t6 = prepro_beta('그게 천재이시다.', lis_beta_ef, lis_tag_last, lis_end_2low)

['NNG', 'VCP', 'EP', 'EF', 'SF']


In [11]:
t5

['NP+JKS', 'NNG/VCP/EP/NULL/SF']

In [12]:
'VCP/NULL' in t5[-1]

False

In [13]:
print(prepro_beta('짜요짜요.', lis_beta_ef, lis_tag_last, lis_end_2low))

['VV', 'EC', 'VV', 'EF', 'SF']
(['짜요짜요.'], ['VV/EC/VV/EF/SF'], [[0, 2, 4, 6, 8, 9]], ['ㅉㅏㅇㅛㅉㅏ.'], ['VV/EC/VV/NULL/SF'], ['ㅇㅛ'])


In [14]:
def prepro_beta_02(input, lis_ef, tag_last, lis_w_last, lis_w_last_not):
    data_w, data_t, lis_ind = prepro(input)
    
    last_words = []

    data_w_jamo = []

    data_t_after = []
    
    lis_target_ind = []
    
    for i in data_w:
        jam_ele = Jamodealer(i)
        ele = ''
        for j in jam_ele.jamo:
            ele = ele+j
        data_w_jamo.append(ele)
    
    for i in range(len(data_t)):
        #if i<len(data_t)-1:
        if i<len(data_t):
            lis_res = []
            for ind in range(len(lis_ind[i])-1):
                lis_res.append(data_w_jamo[i][lis_ind[i][ind]:lis_ind[i][ind+1]])

        
        
                
        if 'EF' in data_t[i] or 'UNKNOWN' in data_t[i]:
            elements = data_t[i].split('/')
            
            flag = 0

            for j in range(len(elements)):
                
                flag_end = 0
                
                for end in lis_w_last_not:
                    if end in lis_res[j]:
                        flag_end = 1
                        break
                
                
                
                if elements[j] in lis_ef and flag_end==0: #and j == len(elements)-1:
                    elements[j] = 'NULL'
                    
                    last_words.append(data_w_jamo[i][lis_ind[i][j]:lis_ind[i][j+1]])
                    lis_res[j]=''
                    
                    lis_target_ind.append(i)
                    
                elif 'EF' in elements[j] and flag_end==0:# + EF를 처리하는 부분이므로 + EF 만을 마지막에서 처리한다.

                    for jam in lis_w_last:
                        if len(lis_res[j])>=len(jam):
                            print(lis_res)

                            res_out_punc = lis_res[:lis_ind[i][-2]][0]

                            #if lis_res[j].find(jam)!=-1:
                            if res_out_punc[-len(jam):]==jam:
                                
                                lis_target_ind.append(i)
                                
                                last_words.append(jam)
                                lis_res[j] = lis_res[j].replace(jam, '', 1)

                                for k in tag_last:

                                    if k in elements[j]:
                                        
                                        if '+' in elements[j]:
                                            
                                            ind = elements[j].index('+'+k)
                                            elements[j] = elements[j][:ind]
                                break
                                
                            elif lis_w_last.index(jam)==len(lis_w_last)-1:#new
                                for k in tag_last:

                                    if k in elements[j]:
                                        
                                        if '+' in elements[j]:
                                            
                                            ind = elements[j].index('+'+k)
                                            elements[j] = elements[j][:ind]
                                            
                                lis_target_ind.append(i)
                                last_words.append('')
                                break

            data_w_jamo[i] = ''.join(lis_res)

            elements_post = '/'.join(elements)
            data_t_after.append(elements_post)
            
        elif 'EC/SF' in data_t[i]:
            
            elements = data_t[i].split('/')
            
            flag = 0
            for j in range(len(elements)):
                
                flag_end = 0
                
                for end in lis_w_last_not:
                    if end in lis_res[j]:
                        flag_end = 1
                        break
                        
                if flag_end==0 and i not in lis_target_ind:
                    lis_target_ind.append(i)
                    last_words.append('')
                
            data_t_after.append(data_t[i])
            
        else:
            data_t_after.append(data_t[i])
            
        
        #################################################################
        
        
        lis_normal = []
    
        for i in data_w_jamo:
            jam_n = Jamodealer(i)
            lis_normal.append(jam_n.make_one())
    
#         lis_target_ind = []

#         word_test_pre, tag_test_pre, ind_test_pre = prepro(input)
    
    
#         for i in range(len(lis_normal)):
#             if lis_normal[i]!=data_w[i]:
#                 lis_target_ind.append(i)
        
        
        
        
        
    return data_w, data_t, lis_ind, data_w_jamo, data_t_after, last_words, lis_target_ind
                        
                    
   

In [15]:
def prepro_beta_03(input, lis_ef, tag_last, lis_w_last, lis_w_last_not):
    data_w, data_t, lis_ind = prepro(input)
    
    last_words = []

    data_w_jamo = []

    data_t_after = []
    
    lis_target_ind = []
    
    for i in data_w:
        jam_ele = Jamodealer(i)
        ele = ''
        for j in jam_ele.jamo:
            ele = ele+j
        data_w_jamo.append(ele)
    
    for i in range(len(data_t)):
        #if i<len(data_t)-1:
        if i<len(data_t):
            lis_res = []
            for ind in range(len(lis_ind[i])-1):
                lis_res.append(data_w_jamo[i][lis_ind[i][ind]:lis_ind[i][ind+1]])

        
        if '/SF' in data_t[i] or '/SV' in data_t[i]:
            if 'EF' in data_t[i] or 'UNKNOWN' in data_t[i]:
                elements = data_t[i].split('/')
            
                flag = 0

                for j in range(len(elements)):
                
                    flag_end = 0
                
                    for end in lis_w_last_not:
                        if end in lis_res[j]:
                            flag_end = 1
                            break
                
                
                
                    if elements[j] in lis_ef and flag_end==0: #and j == len(elements)-1:
                        elements[j] = 'NULL'
                    
                        last_words.append(data_w_jamo[i][lis_ind[i][j]:lis_ind[i][j+1]])
                        lis_res[j]=''
                    
                        lis_target_ind.append(i)
                    
                    elif 'EF' in elements[j] and flag_end==0:# + EF를 처리하는 부분이므로 + EF 만을 마지막에서 처리한다.

                        for jam in lis_w_last:
                            if len(lis_res[j])>=len(jam):
                                print(lis_res)

                                res_out_punc = lis_res[:lis_ind[i][-2]][0]

                                #if lis_res[j].find(jam)!=-1:
                                if res_out_punc[-len(jam):]==jam:
                                
                                    lis_target_ind.append(i)
                                
                                    last_words.append(jam)
                                    lis_res[j] = lis_res[j].replace(jam, '', 1)

                                    for k in tag_last:

                                        if k in elements[j]:
                                        
                                            if '+' in elements[j]:
                                            
                                                ind = elements[j].index('+'+k)
                                                elements[j] = elements[j][:ind]
                                    break
                                
                                elif lis_w_last.index(jam)==len(lis_w_last)-1:#new
                                    for k in tag_last:

                                        if k in elements[j]:
                                        
                                            if '+' in elements[j]:
                                            
                                                ind = elements[j].index('+'+k)
                                                elements[j] = elements[j][:ind]
                                            
                                    lis_target_ind.append(i)
                                    last_words.append('')
                                    break

                data_w_jamo[i] = ''.join(lis_res)

                elements_post = '/'.join(elements)
                data_t_after.append(elements_post)
            #######################################
            
            data_t_after.append(data_t[i])
            
        elif 'EC/SF' in data_t[i]:
            
            elements = data_t[i].split('/')
            
            flag = 0
            for j in range(len(elements)):
                
                flag_end = 0
                
                for end in lis_w_last_not:
                    if end in lis_res[j]:
                        flag_end = 1
                        break
                        
                if flag_end==0 and i not in lis_target_ind:
                    lis_target_ind.append(i)
                    last_words.append('')
                
            data_t_after.append(data_t[i])
            
        else:
            data_t_after.append(data_t[i])
            
        
        #################################################################
        
        
        lis_normal = []
    
        for i in data_w_jamo:
            jam_n = Jamodealer(i)
            lis_normal.append(jam_n.make_one())
    
#         lis_target_ind = []

#         word_test_pre, tag_test_pre, ind_test_pre = prepro(input)
    
    
#         for i in range(len(lis_normal)):
#             if lis_normal[i]!=data_w[i]:
#                 lis_target_ind.append(i)
        
        
        
        
        
    return data_w, data_t, lis_ind, data_w_jamo, data_t_after, last_words, lis_target_ind
                        
                    
   

In [16]:
def prepro_beta_04(input, lis_ef, tag_last, lis_w_last, lis_w_last_not):
    data_w, data_t, lis_ind = prepro(input)
    
    last_words = []

    data_w_jamo = []

    data_t_after = []
    
    lis_target_ind = []
    
    for i in data_w:
        jam_ele = Jamodealer(i)
        ele = ''
        for j in jam_ele.jamo:
            ele = ele+j
        data_w_jamo.append(ele)
    
    for i in range(len(data_t)):
        #if i<len(data_t)-1:
        if i<len(data_t):
            lis_res = []
            for ind in range(len(lis_ind[i])-1):
                lis_res.append(data_w_jamo[i][lis_ind[i][ind]:lis_ind[i][ind+1]])

        
        if 'EF/SF' in data_t[i] or 'EF/SV' in data_t[i] or 'UNKNOWN/SF' in data_t[i] or 'UNKNOWN/SV' in data_t[i]:
            if 'EF/SF' in data_t[i] or 'UNKNOWN' in data_t[i]:
                elements = data_t[i].split('/')
            
                flag = 0

                for j in range(len(elements)):
                
                    flag_end = 0
                
                    for end in lis_w_last_not:
                        if end in lis_res[j]:
                            flag_end = 1
                            break
                
                
                
                    if elements[j] in lis_ef and flag_end==0: #and j == len(elements)-1:
                        elements[j] = 'NULL'
                    
                        last_words.append(data_w_jamo[i][lis_ind[i][j]:lis_ind[i][j+1]])
                        lis_res[j]=''
                    
                        lis_target_ind.append(i)
                    
#                         elements_post = '/'.join(elements)
#                         data_t_after.append(elements_post)
                        
                    elif 'EF' in elements[j] and flag_end==0:# + EF를 처리하는 부분이므로 + EF 만을 마지막에서 처리한다.

                        for jam in lis_w_last:
                            if len(lis_res[j])>=len(jam):
                                
                                res_out_punc = lis_res[:lis_ind[i][-2]][j]

                                if res_out_punc[-len(jam):]==jam:
                                
                                    lis_target_ind.append(i)
                                
                                    last_words.append(jam)
                                    lis_res[j] = lis_res[j].replace(jam, '', 1)

                                    for k in tag_last:

                                        if k in elements[j]:
                                        
                                            if '+' in elements[j]:
                                            
                                                ind = elements[j].index('+'+k)
                                                elements[j] = elements[j][:ind]
                                    break
                                
                                elif lis_w_last.index(jam)==len(lis_w_last)-1:#new
                                    for k in tag_last:

                                        if k in elements[j]:
                                        
                                            if '+' in elements[j]:
                                            
                                                ind = elements[j].index('+'+k)
                                                elements[j] = elements[j][:ind]
                                            
                                    lis_target_ind.append(i)
                                    last_words.append('')
                                    break

                
                elements_post = '/'.join(elements)
                data_t_after.append(elements_post)
                
                data_w_jamo[i] = ''.join(lis_res)

                #elements_post = '/'.join(elements)
                #data_t_after.append(elements_post)
                    
            #######################################
            
            #data_t_after.append(data_t[i])
            
        elif 'EC/SF' in data_t[i]:
            
            elements = data_t[i].split('/')
            
            flag = 0
            for j in range(len(elements)):
                
                flag_end = 0
                
                for end in lis_w_last_not:
                    if end in lis_res[j]:
                        flag_end = 1
                        break
                        
                if flag_end==0 and i not in lis_target_ind:
                    lis_target_ind.append(i)
                    last_words.append('')
            print(data_t[i])
            data_t_after.append(data_t[i])
            
        else:
            data_t_after.append(data_t[i])
            
        
        #################################################################
        
        
        lis_normal = []
    
        for i in data_w_jamo:
            jam_n = Jamodealer(i)
            lis_normal.append(jam_n.make_one())
    
#         lis_target_ind = []

#         word_test_pre, tag_test_pre, ind_test_pre = prepro(input)
    
    
#         for i in range(len(lis_normal)):
#             if lis_normal[i]!=data_w[i]:
#                 lis_target_ind.append(i)
        
        
        
        
        
    return data_w, data_t, lis_ind, data_w_jamo, data_t_after, last_words, lis_target_ind
                        
                    
   

In [17]:
print(prepro_beta_03('백화점은 문을 닫았고 일자리는 구하기 힘들어졌다.', lis_beta_ef, lis_tag_last, lis_end_2low, lis_end))

(['백화점은', '문을', '닫았고', '일자리는', '구하기', '힘들어졌다.'], ['NNG/JX', 'NNG/JKO', 'VV/EP/EC', 'NNG/JX', 'VV/ETN', 'VA/EC/VX+EP/EF/SF'], [[0, 8, 11], [0, 3, 6], [0, 3, 6, 8], [0, 7, 10], [0, 4, 6], [0, 6, 8, 11, 13, 14]], ['ㅂㅐㄱㅎㅘㅈㅓㅁㅇㅡㄴ', 'ㅁㅜㄴㅇㅡㄹ', 'ㄷㅏㄷㅇㅏㅆㄱㅗ', 'ㅇㅣㄹㅈㅏㄹㅣㄴㅡㄴ', 'ㄱㅜㅎㅏㄱㅣ', 'ㅎㅣㅁㄷㅡㄹㅇㅓㅈㅕㅆ.'], ['NNG/JX', 'NNG/JKO', 'VV/EP/EC', 'NNG/JX', 'VV/ETN', 'VA/EC/VX+EP/NULL/SF', 'VA/EC/VX+EP/EF/SF'], ['ㄷㅏ'], [5])


In [18]:
origin_w,origin_t, origin_ind, cont_w, con_t, ending, i = prepro_beta_04('어디서 노세요?', lis_beta_ef, lis_tag_last, lis_end, lis_end_2low)

In [19]:
cont_w[-1][-1]

'?'

In [20]:
cont_w

['ㅇㅓㄷㅣㅅㅓ', 'ㄴㅗ?']

In [21]:
######높임말 -> 반말
#현재 만들어진 것은 EF만 잘라낼 것이다. 
#원래 ef사전에 mapping 되는 것을 찾아낸다.
# EF_1 = {
#     ['ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']:['ㄷ','ㅏ'],
#     ['ㅅ','ㅡ','ㅂ','ㄴ','ㅣ','ㄷ','ㅏ']:['ㄷ','ㅏ']
# }
#python dictionary로 접근
#종결어미 처리
EF = {
    ###하십시오체###
    #평서문
    'ㅂㄴㅣㄷㅏ': 'ㄷㅏ',
    'ㅅㅡㅂㄴㅣㄷㅏ':'special',
    'ㅇㅗㄹㅅㅣㄷㅏ':'ㄷㅏ', #**
    'ㅂㅈㅣㅇㅛ':'지', #**
    #의문문
    'ㅅㅡㅂㄴㅣㄲㅏ':'special',
    'ㅅㅣㅂㄴㅣㄲㅏ':'special1', #EP+EF
    #명령법
    'ㅇㅡㅅㅔㅇㅛ': 'special',
    'ㅅㅔㅇㅛ':'special1',
    #청유법
    
    #명령인용법
    
    ###하오체###
    
    ###해요체###
    'ㅇㅓㅇㅛ':'ㅇㅓ',
    'ㅇㅏㅇㅛ':'ㅇㅏ',
    
}
#'ㄹ'규칙 활용 -> ㄹ 규칙 활용이 일어나는 동사들을 최대한 모아두고, 만약 하나의 단어에 여러 의미가 담긴다면?
EF_SaeYo_R= {
    'ㄱㅜ':'ㄹ',
    'ㄴㅗ':'ㄹ',
    'ㄴㅏ':'ㄹ',
    'ㄷㅗ':'ㄹ',
    'ㄷㅡ':'ㄹ',
    'ㄷㅏ':'ㄹ',
    'ㄷㅜ':'ㄹ',
    'ㅂㅜ':'ㄹ',
    'ㄲㅗ':'ㄹ',
    'ㅁㅣ':'ㄹ',
    'ㅁㅜ':'ㄹ',
    #'ㅂㅗㅍㅜ':'ㄹ', #error predicate 수정
    'ㅂㅜ':'ㄹ',
    'ㅅㅡ':'ㄹ',
    'ㄸㅓ':'ㄹ',
    
}

####EXAMPLE####
tag = 'ㅅㅔㅇㅛ'
# sentence = ['ㄱㅡㄴㅡㄴ', 'ㅅㅏㄴㅇㅡㄹ', 'ㅇㅗㄹㅡ']
# tagList = ['NP/JX', 'NNG/JKO', 'VV/']

# tag2 = 'ㅅㅔㅇㅛ'
# sentence2 = ['ㄱㅡㄴㅡㄴ', 'ㅂㅐㄹㄱㅗㅍㅡ']
# tagList2 = ['NP/JX', 'VA/']

####EXAMPLE####

## 'ㅏ', 'ㅗ' 처리
def convertSpecialCase_AhOh(sentence):
    #print(sentence[-1][-2:])
    print(sentence)
    ## 수정할 필요 있음!
    if sentence[-1][-2:] == 'ㅍㅡ' or sentence[-1][-2:] == 'ㅃㅡ':
        if sentence[-1][-4:-2].find('ㅏ') !=-1 or sentence[-1][-4:-2].find('ㅗ')!=-1:
            return 'ㅏ'
        else:
            return 'ㅓ'
    elif sentence[-1][-2:].find('ㅏ') !=-1 or sentence[-1][-2:].find('ㅗ') !=-1:
        return 'ㅇㅏ'
    else:
        return 'ㅇㅓ'

def convertSpecialCase_SaeYo(sentence, tag):
    result = ''
    end_EF=''
    final=''
    predicate = sentence[-1][-2:]
    #print(predicate)
    # 'ㄹ'규칙 활용
    if predicate in EF_SaeYo_R:
        result= EF_SaeYo_R[predicate]
        #sentence[-1] += result
        ##'아' 또는 '어' 로 처리
        end_EF = convertSpecialCase_AhOh(sentence)
        #sentence.append(end_EF)
        final = result +end_EF
        print(final)
    #'르' 불규칙 활용
    elif predicate =='ㄹㅡ':
        # 용언 종성에 ㄹ이 있다면
        sentence[-1] = sentence[-1].replace('ㄹㅡ','')
        end_EF = convertSpecialCase_AhOh(sentence)
        end_EF = end_EF[-1]
        if sentence[-1].find('ㄹ') != -1:
            final = 'ㄹ'+end_EF
        else:
            final  = 'ㄹㄹ'+end_EF
        
    #'우' 불규칙 활용
    # '푸'를 제외한 다른 'ㅜ'는 'ㅓ'와 결합
    elif predicate.find('ㅍㅜ') !=-1:
        #sentence[-1] = sentence[-1].replace('ㅜ','ㅓ')
        sentence[-1] = sentence[-1].replace('ㅜ','')
        final = 'ㅓ'
        #final = sentence[-1][-1:]
    #'오' 불규칙 활용(고려하지 않을 수 있음)
    #'하' 불규칙 활용
    elif predicate.find('ㅎㅏ') !=-1:
        #sentence[-1] = sentence[-1].replace('ㅏ','ㅐ')
        sentence[-1] = sentence[-1].replace('ㅏ','')
        final = 'ㅐ'
    #활용이 안되었던 용언 처리
    else:
        if predicate.find('ㅡ') !=-1:
            end_EF = convertSpecialCase_AhOh(sentence)
            #sentence[-1] = sentence[-1].replace('ㅡ',end_EF)
            sentence[-1] = sentence[-1].replace('ㅡ','')
            final = end_EF[-1]
        ## 수정할 필요 있음!!
        elif predicate.find('ㅗ') !=-1:
            #sentence[-1] = sentence[-1].replace('ㅗ','ㅘ')
            sentence[-1] = sentence[-1].replace('ㅗ','')
            final = 'ㅘ'
        elif predicate.find('ㅜ') !=-1:
            #sentence[-1] = sentence[-1].replace('ㅜ','ㅝ')
            sentence[-1] = sentence[-1].replace('ㅜ','') 
            final = 'ㅝ'
        elif predicate.find('ㅣ') !=-1:
            #sentence[-1] = sentence[-1].replace('ㅣ','ㅕ')
            sentence[-1] = sentence[-1].replace('ㅣ','')
            final = 'ㅕ'
        else:
            final = convertSpecialCase_AhOh(sentence)
            #sentence = sentence.append(final)
            if  predicate.find('ㅏ') !=-1:
                final = ''
                return final
            return final
    return final
    

##if Verb & adjective
def treatVerb_Adjective(sentence, ending):
    re_value =''
    for key in EF:
        if ending == key:
            re_value = EF[key]
            #습니다, 나요, 으세요 case
            if re_value == 'special':
                re_value = convertSpecialCase_AhOh(sentence)
                #ㅗ,ㅜ 이면 ㅘ, ㅝ로 결합할 것
                #print(re_value)
            #-세요
            elif re_value == 'special1':
                #print(sentence)
                re_value = convertSpecialCase_SaeYo(sentence, ending)
    # print(sentence)
    return re_value

punc = cont_w[-1][-1]
cont_w[-1] = cont_w[-1][:-1]
print(cont_w)
print(ending)
end = ''.join(ending)
re_value = treatVerb_Adjective(cont_w, end)
print(re_value)
cont_w[-1] += re_value
#print(sentence)

res_space = []
for i in range(len(cont_w)):
    res_space.append(cont_w[i])
    if i==len(cont_w)-1:
        break
    res_space.append(' ')
jam_q = Jamodealer(res_space)

result = jam_q.make_one()
print(result + punc)
#VX는 따로 처리해야 함

# # key_list[0]
# def ChangeMorphemeToLow(tag, sentence):
#     #for문으로 ef list 탐색 후 일치한다면 변환 후 return 
#     #key_list = EF_1.keys()
#     for key in EF:
#         if tag == key:
#             if EF
#             tag = EF_1[key]
#     print(tag)
#     return tag

    
    

['ㅇㅓㄷㅣㅅㅓ', 'ㄴㅗ']
['ㅅㅔㅇㅛ']
['ㅇㅓㄷㅣㅅㅓ', 'ㄴㅗ']
ㄹㅇㅏ
ㄹㅇㅏ
어디서 놀아?


In [22]:
def convert_an(stc, tag, ex):
    
    ind_point = -1
    point = ''

#     for i in P_LIST:
#         if stc.find(i) !=-1:
#             point = i
#             break
            
    for i in range(len(stc)):
        if stc[i] in P_LIST:
            point = stc[i]
            break
            
    if point in P_LIST:
        ind_point = stc.index(point)
    
    r_word = ''
    r_pun = ''
    
    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = stc
    
    if ex=='ㄹㅏ':
        return r_word+'ㅇㅛ'+r_pun
    elif ex=='ㄷㅏㄷㅓㄹㅏ':
        return r_word+'ㄷㅐㅇㅛ'+r_pun
    elif ex=='ㄷㅓㄹㅏ':
        wd = ''
        if 'ㄷㅏ' in r_word:
#             print('dd')
            return r_word[:-1]+'ㅐㅇㅛ'+r_pun
        if r_word[-1] in jongsung_list:
            wd = r_word[-2]
        else:
            wd = r_word[-1]
        if wd=='ㅗ':
            return r_word+'ㅏㅇㅛ'+r_pun
        elif wd=='ㅜ':
            return r_word+'ㅓㅇㅛ'+r_pun
        elif wd=='ㅏ':
            return r_word+'ㅇㅏㅇㅛ'+r_pun
        else:
            return r_word+'ㅇㅓㅇㅛ'+r_pun
    elif ex=='ㄴㅣ' or ex =='ㄴㅡㄴㄱㅏ' or ex =='ㅇㅡㄴㄱㅏ':
        return r_word+'ㄴㅏㅇㅛ'+r_pun
    elif ex=='ㅇㅑ':
        return r_word+'ㅇㅔㅇㅛ'+r_pun
#     elif ex=='ㄷㅓㄹㅏ':
#         if 'ㄷㅏ' in stc:
#             return stc[:-1]+'ㅔㅇㅛ'
#         return stc+'ㄷㅔㅇㅛ'
    elif ex=='ㅈㅣ':
        return r_word+'ㅈㅛ'+r_pun
    else:
        return r_word+ex+'ㅇㅛ'+r_pun
    
def convert_da(stc, tag, ex):
    #print(tag)
    isVcp = tag[-6:]
#     print(isVcp)
    ind_point = -1
    point = ''
    for i in range(len(stc)):
        if stc[i] in P_LIST:
            point = stc[i]
            break
            
    if point in P_LIST:
        ind_point = stc.index(point)
    
    r_word = ''
    r_pun = ''
    
    if ind_point!=-1:
        r_word = stc[:ind_point]
        r_pun = stc[ind_point:]
    else:
        r_word = stc
    
    try:
    
        jongsung = r_word[-1]
    
    except:
        
        return ex
        
    if jongsung in jongsung_list:
        final = r_word +'ㅅㅡㅂㄴㅣㄷㅏ'+r_pun
    else:
        if isVcp.find('VCP') != -1:
            print('what the')
            final = r_word +'ㅇㅣㅂㄴㅣㄷㅏ'+r_pun
        else:
            final = r_word +'ㅂㄴㅣㄷㅏ'+r_pun
    #print(final)
    return final

In [23]:
class Changer(object):
    
    def make_end_h(self, stc, tag, ex):

#         print(ex)
        if 'ㄷㅏ'== ex[-2:]:
            return convert_da(stc, tag, ex)
        else:
            return convert_an(stc, tag, ex)
    

    
    def to_high(self, input):
        result = input
        space_list = rememberSpace(input,' ')
        
        test_w, test_t = to2lists(result)
        pre_w, pre_t, pre_ind = prepro(result)
        
        data_w, data_t, lis_i, lis_w, lis_t, off_word, lis_target_ind = prepro_beta_04(result, lis_beta_ef, lis_tag_last, lis_end_2low, lis_end)

        
#         lis_w, lis_t, lis_i, w_last, t_last, off_word, lis_normal, lis_target_ind = prepro_after(result, lis_end_2low)
#         space_location = convertSpace(space_list, pre_w)
        space_location = convertSpace(space_list, data_w)
#         lis_target_final = []
        
        if len(lis_target_ind)!=0:

            jam = Jamodealer(data_w)

            for i in range(len(lis_target_ind)):
#                 print(lis_target_ind)
#                 print(i)
#                 print(lis_w[10])
#                 print(lis_t[10])
#                 print(off_word)
#                 print(lis_target_ind[0])
                new_end = self.make_end_h(lis_w[lis_target_ind[i]], lis_t[lis_target_ind[i]], off_word[i])
                jam.jamo[lis_target_ind[i]] = new_end
            
            
#             jam.jamo.append(w_last)
            union(space_location, jam.jamo)
            return jam.make_one()

        return input


    
        
    def processText(self,stc):
        result = stc
        
        flag = 0
        if result[-1]=='\n':
            result = result.replace('\n','')
        
        num = 0
        while 1:
            if result[-1-num]!=' ':
                break
            else:
                num = num+1
                
        
                
        if num==0:
            rere = result
        else:
            rere = result[:-num]
            
        
        r_pun = ''
        r_word = rere
        while True:
            if r_word[-1] in SV_LIST:
                r_pun = r_pun+r_word[-1]
                r_word = r_word[:-1]
            else:
                break
        
        
        num_space = 0
        for i in r_word:
            if i==' ':
                num_space = num_space+1
            else:
                break
            
        if num_space!=0:
            r_word = r_word[num_space:]

#         cc = ch.processText(line)
    
        plus = ''
        for s in range(num_space):
            plus = plus+' '
    
        if r_word[-1] =='?' or r_word[-1] =='.' or r_word[-1] =='!' or r_word[-1] =='\"':
            r_word = r_word
        else:
            r_word = r_word+'.'
            flag = 1
        
        res = self.to_high(r_word)
        
        r_word = plus+r_word
        res = plus+res
        
        
        
#         if r_word[-1] =='?' or r_word[-1] =='.' or r_word[-1] =='!' or r_word[-1] =='\"':
#             r_word = r_word
#         else:
#             r_word = r_word+'.'
#             flag = 1
        
#         res = self.to_high(r_word)
        
        if flag ==1:
            res = res[:-1]
        
        #spacing = Spacing()
        #res = spacing(res)
        return res+r_pun[::-1]

In [24]:
ch = Changer()

In [25]:
ch.to_high('9시? 9시 15분에 병원 예약이 있는데. 좀 태워다 줄 수 있겠지?')

'9시? 9시 15분에 병원 예약이 있는데요. 좀 태워다 줄 수 있겠죠?'

In [26]:
f0 = open('./hgu_clean.kr', 'r', encoding = 'utf-8')
f1 = open('./new_high_01.kr', 'w',encoding = 'utf-8')

flag = 0

num = 0
number = 0
ch = Changer()

while True:
    flag = 0
    line = f0.readline()
    if not line: break
    if line[-1]=='\n':
        line = line.replace('\n','')
    
    num_space = 0
    for i in line:
        if i==' ':
            num_space = num_space+1
        else:
            break
            
    if num_space!=0:
        line = line[num_space:]

    cc = ch.processText(line)
    
    plus = ''
    for s in range(num_space):
        plus = plus+' '
    
    line = plus+line
    cc = plus+cc
    
    if number<100000:
        if line!=cc:
            f1.write(line)
            f1.write('\n')
            f1.write(cc)
            f1.write('\n\n')
        num = num+1
    else:
        break
    print(number)
    number = number+1
f0.close()
f1.close()

FileNotFoundError: [Errno 2] No such file or directory: './hgu_clean.kr'

In [146]:
ch.processText('걱정 말아요. 내가 비디오 녹화해 놨으니까.')

VX+EP+EC/SF


'걱정 말아요. 내가 비디오 녹화해 놨으니까요.'